In [1]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

c:\Users\user\anaconda3\envs\nof\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터호출

In [5]:
## 키워드 작업한 데이터 불러오기
import pandas as pd
import torch

df_fraud = pd.read_csv('data/df_fraud_keyword.csv', index_col=False)

In [3]:
## DB에 저장한 키워드작업된 데이터 가져오기(사기기)
import mysql.connector
import pandas as pd
from db_info import user, password, host, port, database

db_connection = mysql.connector.connect(
    host = host,
    user = user,
    password = password,
    database = database,
    port = port
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_keyword")

result = cursor.fetchall()

df_injury = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

In [4]:
df_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4187 entries, 0 to 4186
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4187 non-null   int64 
 1   사건명            4186 non-null   object
 2   사건번호           4187 non-null   object
 3   선고일자           4187 non-null   object
 4   법원명            4187 non-null   object
 5   사건종류명          4187 non-null   object
 6   판례상세링크         4187 non-null   object
 7   판례정보일련번호       4187 non-null   int64 
 8   사건번호.1         4187 non-null   object
 9   선고             4187 non-null   object
 10  판결유형           4187 non-null   object
 11  판시사항           3415 non-null   object
 12  판결요지           3026 non-null   object
 13  참조조문           3390 non-null   object
 14  참조판례           1924 non-null   object
 15  판례내용           4187 non-null   object
 16  판례내용_상단        4187 non-null   object
 17  판례내용_이유        4187 non-null   object
 18  판례내용_이유_전처리    4187 non-null

In [5]:
df_injury.drop(columns=['판례정보일련번호','사건번호.1'], inplace=True)
df_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4187 entries, 0 to 4186
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4187 non-null   int64 
 1   사건명            4186 non-null   object
 2   사건번호           4187 non-null   object
 3   선고일자           4187 non-null   object
 4   법원명            4187 non-null   object
 5   사건종류명          4187 non-null   object
 6   판례상세링크         4187 non-null   object
 7   선고             4187 non-null   object
 8   판결유형           4187 non-null   object
 9   판시사항           3415 non-null   object
 10  판결요지           3026 non-null   object
 11  참조조문           3390 non-null   object
 12  참조판례           1924 non-null   object
 13  판례내용           4187 non-null   object
 14  판례내용_상단        4187 non-null   object
 15  판례내용_이유        4187 non-null   object
 16  판례내용_이유_전처리    4187 non-null   object
 17  판례내용_이유_불용어제거  4187 non-null   object
 18  클러스터           4187 non-null

In [57]:
df_fraud[df_fraud['사건명'].isnull()]

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,판례상세링크,판례정보일련번호,선고,판결유형,...,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,판례내용_이유_전처리,판례내용_이유_불용어제거,클러스터,키워드
3478,71592,None,2001노3042,2002.04.12,서울고등법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,71592,선고,판결：상고기각,...,[1] 특정범죄가중처벌등에관한법률 제5조의2 제4항에서 영리의 목적...,"\n[1]\n\n 형법 제288조 제1항\n,\n\n ...",None,【피고인】 피고인【항소인】 피고인【변호인】 변호사 이수형 외 1인【원심판결】 ...,【피고인】 피고인【항소인】 피고인【변호인】 변호사 이수형 외 1인【원심판결】 ...,【이유】1. 항소이유의 요지가. 사실오인피고인은 무허가 직업소개소를 운영하면서 원심...,{'이유': '1. 항소이유의 요지가. 사실오인피고인은 무허가 직업소개소를 운영하면...,사실오인 무허가 직업 소개 운영 데리 도록 망하여 유인 의사 과장 하였을 이미 다...,7,"['지급', '계약', '사기죄', '채권', '신청', '회사', '공사', '의..."
3793,105119,None,86다카1147,1989.04.25,대법원,민사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,105119,선고,판결,...,가. 계약이 합의해제된 경우에는 그 해제시에 당사자 일방이 상대방에게 손해배상을 하...,가.\n \n 민법 제551조 / 나. \n 제664조<br/>,가. \n \n대법원 1960.10.6. 선고 4293민상275 판결 ...,【원고(반소피고)】 상고인 곽태훈【피고(반소원고)】 피상고인 덕화공영주식...,【원고(반소피고)】 상고인 곽태훈【피고(반소원고)】 피상고인 덕화공영주식...,"【이 유】 1. 상고이유 제1점을 본다. 원심판결 이유에 의하면, 원심은 ...","{'이유': '1. 상고이유 제1점을 본다. 원심판결 이유에 의하면, 원심은 원고(...",본다 손해배상 1983.5 덕화 주식회사 소원 회사 와의 소유 대지 건축 지하 지상...,2,"['회사', '주식', '주식회사', '자금', '대출', '거래', '투자', '..."


In [67]:
## DB에 저장한 키워드작업된 데이터 가져오기
import mysql.connector
import pandas as pd
from db_info import user, password, host, port, database

db_connection = mysql.connector.connect(
    host = host,
    user = user,
    password = password,
    database = database,
    port = port
)

cursor = db_connection.cursor()

cursor.execute("select * from labor_keyword")

result = cursor.fetchall()

df_labor = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

In [9]:
df_labor.info()

NameError: name 'df_labor' is not defined

# 승민님 BERT모델(판시사항)

In [1]:
print(df_fraud['판시사항'].iloc[10])


NameError: name 'df_fraud' is not defined

In [ ]:
df_fraud['모델용_문장'] = "[CLS]" + df_fraud['판시사항'] + "[SEP]"

In [ ]:
df_fraud['모델용_문장'].iloc[1]

'[CLS] 배심원이 참여하는 형사재판, 즉 국민참여재판을 거쳐 제1심법원이 배심원의 만장일치 무죄평결을 받아들여 피고인에 대하여 무죄판결을 선고한 경우, ‘증거의 취사 및 사실의 인정’에 관한 제1심법원의 판단은 한층 더 존중될 필요가 있는지 여부(적극) 및 이때 제1심법원의 무죄판결에 대한 항소심에서의 추가적이거나 새로운 증거조사의 범위[SEP]'

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from kobert_transformers import get_tokenizer

bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

# Define your model
model = distilbert_model  # 모델 선언
tokenizer = get_tokenizer()  # 토크나이저 선언

for i in range(10):
    sentences = []
    sentences.append(df_fraud['모델용_문장'].iloc[i])


# 각 문장을 토큰화, id화
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_sentences]

# sequence를 같은 길이로 padding
max_length = max(len(ids) for ids in tokenid_list)
padded_tokenid_list = [ids + [0] * (max_length - len(ids)) for ids in tokenid_list]

# Longtensor로 dtype변경
tokenid_tensor = torch.LongTensor(padded_tokenid_list)

# Print the tensor
print(tokenid_tensor)

# 추론
tensor_output = model(tokenid_tensor)  # 모델에 위의 전처리 과정을 거친 자연어를 input으로 넣는다
print(tensor_output[0].shape)  # Output shape
print(tensor_output[0])  # Output 확인

# 출력은 (batchsize, 토큰id의 길이, 768). 여기서 768은 임베딩 공간의 크기

c:\Users\user\anaconda3\envs\nof\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--monologg--kobert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tensor([[   2, 3893, 5478, 6335, 4128,  150, 7253, 4128,   93, 7846, 6903, 4092,
         7828, 4071, 7202, 3480, 5384, 7096, 4596, 5898, 3162, 2682, 3893,  517,
         7656, 5781, 7088, 4955, 2587, 1528, 6006,  833, 2322, 4128,  225, 7253,
         4128,  116, 7925, 6896, 1835, 4466, 5889, 7318, 3310,   18, 7202, 5543,
           40,    3]])
torch.Size([1, 50, 768])
tensor([[[ 0.0527,  0.2446,  0.1086,  ..., -0.5052, -0.5448,  0.1624],
         [ 0.6586,  0.0192,  0.1596,  ...,  0.0030, -0.1197,  0.2213],
         [ 0.4585, -0.0808,  0.4904,  ...,  0.1427, -0.2438, -0.2900],
         ...,
         [ 0.6087, -0.0853,  0.3526,  ..., -0.3582, -0.3065, -0.0323],
         [ 0.1492,  0.0111,  0.4568,  ..., -0.2438,  0.0845, -0.7004],
         [ 0.0525,  0.2449,  0.1089,  ..., -0.5057, -0.5442,  0.1627]]],
       grad_fn=<NativeLayerNormBackward0>)


# Bert코드

In [7]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#KoBERT모델, 토크나이저 생성
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", return_attention_mask=True)
    with torch.no_grad():
        # outputs = model(**inputs)
        outputs = model(input_ids = inputs['input_ids'], attention_mask=inputs['attention_mask'])
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 판례 내용, 키워드, 판결요지, 판시사항 임베딩 생성
df_injury['판례내용이유임베딩'] = df_injury['판례내용_이유'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_injury['키워드임베딩'] = df_injury['키워드'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_injury['판결요지임베딩'] = df_injury['판결요지'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_injury['판시사항임베딩'] = df_injury['판시사항'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)

# CSV 파일로 저장
df_injury.to_csv('data/df_injury_embedding.csv', index=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#KoBERT모델, 토크나이저 생성
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", return_attention_mask=True)
    with torch.no_grad():
        # outputs = model(**inputs)
        outputs = model(input_ids = inputs['input_ids'], attention_mask=inputs['attention_mask'])
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 판례 내용, 키워드, 판결요지, 판시사항 임베딩 생성
df_fraud['판례내용이유임베딩'] = df_fraud['판례내용_이유'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_fraud['키워드임베딩'] = df_fraud['키워드'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_fraud['판결요지임베딩'] = df_fraud['판결요지'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_fraud['판시사항임베딩'] = df_fraud['판시사항'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)

# CSV 파일로 저장
df_fraud.to_csv('data/df_fraud_embedding.csv', index=False)

In [61]:
df_fraud.drop(columns=['선고일자','법원명','사건종류명','판례상세링크','판례정보일련번호','선고'], inplace=True)

In [62]:
df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4178 entries, 0 to 4177
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4178 non-null   int64 
 1   사건명            4176 non-null   object
 2   사건번호           4178 non-null   object
 3   판결유형           4178 non-null   object
 4   판시사항           3025 non-null   object
 5   판결요지           2479 non-null   object
 6   참조조문           2998 non-null   object
 7   참조판례           1805 non-null   object
 8   판례내용           4178 non-null   object
 9   판례내용_상단        4178 non-null   object
 10  판례내용_이유        4178 non-null   object
 11  판례내용_이유_전처리    4178 non-null   object
 12  판례내용_이유_불용어제거  4178 non-null   object
 13  클러스터           4178 non-null   int64 
 14  키워드            4178 non-null   object
 15  판례내용이유임베딩      4178 non-null   object
 16  키워드임베딩         4178 non-null   object
 17  판결요지임베딩        2479 non-null   object
 18  판시사항임베딩        3025 non-null

In [ ]:
### 진형코드
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수
def encode_sentence(sentence):
    if pd.notnull(sentence):
        inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()
    return None

# 클러스터별 판례 내용과 키워드 임베딩 생성
for cluster_num in range(1, 9):  # 클러스터 1부터 8까지
    cluster_cases = df_inheritance[df_inheritance['클러스터'] == cluster_num]
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판례내용이유임베딩'] = \
        cluster_cases['판례내용_이유'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '키워드임베딩'] = \
        cluster_cases['키워드'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판결요지임베딩'] = \
        cluster_cases['판결요지'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판시사항임베딩'] = \
        cluster_cases['판시사항'].apply(encode_sentence)

# CSV 파일로 저장
df_inheritance.to_csv('df_inheritance_kobertmodel2.csv', index=False)

# 유사도 계산

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_labor.iterrows():
    if row['판례내용이유임베딩'] is not None:
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판례내용이유임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = df_labor.loc[df_labor['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

NameError: name 'df_labor' is not defined